My goal is to ine-tune meta-llama/Meta-Llama-3-8B on timdettmers/openassistant-guanaco.  However, according to 
https://www.reddit.com/r/LocalLLaMA/s/nRlinxXZgp, I need 160GiB VRAM to for a 7B model (at least -- that table comes from
some particular fine-tuning stack and I might be less efficient).  So I'd need at least that if not more for an 8B one.  
That would require renting a machine, or machines.  So can I at least do initial experiments without that?

The 7B:160GiB ratio implies a 22.8 parameters-to-VRAM ratiorequired.  Assuming that's true, I should be able to fine-tune a 1B
model on my 23GiB card, *just*.

Let's see how far we can get.

In [1]:
# This is a 0.5B model so should certainly be trainable on my GPU.
base_model = "Qwen/Qwen1.5-0.5B"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import pipeline

def ask_question(model, prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])
    
ask_question(model, "Who is Leonardo Da Vinci?")

<s>[INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist.

[INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist. [INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist. [INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist. [INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist. [INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C. He is a musician. D. D. D. He is a scientist. [INST] Who is Leonardo Da Vinci? [/INST] A. A. A. He is a painter. B. B. B. He is a sculptor. C. C. C.

Excellent, we got some garbage of the kind you'd expect to get from a base model